In [ ]:
# 导包
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.window import Window
from datetime import datetime,timedelta
import requests 
import re
import json   

In [ ]:
# 读取数据
workshop_address = 'abfss://test-data@dlsaaddpnorth3001.dfs.core.chinacloudapi.cn/test/WZX/03-workshop/workshop_address.csv'
branch_address = 'abfss://test-data@dlsaaddpnorth3001.dfs.core.chinacloudapi.cn/test/WZX/03-workshop/branch_address.csv'
workshop = spark.read.csv(workshop_address,header=True)
branch = spark.read.csv(branch_address,header=True).where(F.col('branch_name') != '"')

In [ ]:
"""todo8: 地理识别"""
def formatted_addr(addr):
    if addr is None:
        return ''
    addr = re.sub(r'[^\u4e00-\u9fa5a-zA-Z0-9]', '', addr)
    addr_key = '1f206129ec240f1cc12be917b9615c0b'
    _url = 'https://restapi.amap.com/v3/geocode/geo?address={}&output=JSON&key={}'.format(addr, addr_key)
    addr_data = '' + "&" + '' + "&" + '' + "&" + '' + "&" + '' + "&" + '0.0,0.0'
    respon = requests.get(_url)
    response = json.loads(respon.content)
    status = response['status']
    info = response['info']
    if status == '1' and info == 'OK':
        count = response['count']
        if count >= '1':
            data = response["geocodes"][0]
            formatted_address = data["formatted_address"]
            country = data["country"]
            province = data["province"]
            city = data["city"]
            district = data["district"]
            location = data["location"]
            addr_data = str(formatted_address) + "&" + str(country) + "&" + str(province) + "&" + str(
                city) + "&" + str(district) + "&" + str(location)
        elif count == '0':
            addr_data = '' + "&" + '' + "&" + '' + "&" + '' + "&" + '' + "&" + '0.0,0.0'
    elif status == '0' and info != 'OK':
        addr_data = '' + "&" + '' + "&" + '' + "&" + '' + "&" + '' + "&" + '0.0,0.0'

    r0 = addr_data.split('&')[0] # address
    r1 = addr_data.split('&')[1] # country
    r2 = addr_data.split('&')[2] # province
    r3 = addr_data.split('&')[3] # city
    r4 = addr_data.split('&')[4] # district
    r5 = addr_data.split('&')[5] # Latitude and longitude
    ra = '|'.join([r0,r5])


    return ra
gd_udf = F.udf(formatted_addr,T.StringType())

In [ ]:
formatted_addr('无锡市广益汽配城A区1#210室')

In [ ]:
df1 = branch.withColumn('all',gd_udf(F.col(' detail_address'))) \   
                   .withColumn('address',F.split(F.col('all'),'\|').getItem(0)) \
                   .withColumn('latitude_and_longitude',F.split(F.col('all'),'\|').getItem(1)) \
                   .withColumnRenamed(' detail_address','detail_address') \
                   .drop('all')

In [ ]:
df2 = workshop.withColumn('all',gd_udf(F.col('detail_address'))) \
              .withColumn('address',F.split(F.col('all'),'\|').getItem(0)) \
              .withColumn('latitude_and_longitude',F.split(F.col('all'),'\|').getItem(1)) \
              .drop('all')

In [ ]:
# save 
save1 = 'abfss://test-data@dlsaaddpnorth3001.dfs.core.chinacloudapi.cn/test/WZX/03-workshop/branch_address.xlsx'
save2 = 'abfss://test-data@dlsaaddpnorth3001.dfs.core.chinacloudapi.cn/test/WZX/03-workshop/workshop_address.xlsx'

# df1.toPandas().to_excel(save1,index=False,header=True)
df2.toPandas().to_excel(save2,index=False,header=True)